In [ ]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import logging
from logging.handlers import TimedRotatingFileHandler

In [3]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747'}

In [4]:
inquiry = Inquiry(market)
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [5]:
def open_inquiry_in_new_tab(self, inquiry):
    href = self.browser.find_element_by_css_selector('a[data-trade-id="'+inquiry['id']+'"]').get_attribute('href')
    self.browser.execute_script("window.open('');")
    self.browser.switch_to.window(self.browser.window_handles[1])
    self.browser.get(href)

inquiry.open_inquiry_in_new_tab = types.MethodType(open_inquiry_in_new_tab, inquiry)

In [6]:
def close_inquiry_and_switch_back(self):
    self.browser.close()
    self.browser.switch_to_window(self.browser.window_handles[0])

inquiry.close_inquiry_and_switch_back = types.MethodType(close_inquiry_and_switch_back, inquiry)

In [25]:
def get_conversation(self):
    html = pq(self.browser.page_source)
    divs = html.find('div.main-content-chat div.item-content-wrap')
    messages = []
    for elem  in divs:
        div = pq(elem)
        if div.hasClass('notification'):
            continue
        div = div.find('div.item-content')
        msg = {}
        if div.hasClass('item-content-left'):
            msg['position'] = 'left'
        elif div.hasClass('item-content-right'):
            msg['position'] = 'right'
        msg['user'] = div.find('.item-base-info span.name').text()
        msg['date'] = div.find('.item-base-info span:last-child').text()
        msg['content'] = div.find('.session-rich-content').text()
        messages.append(msg)
    buyer = {}
    email = html.find('div.main-content-sidebar div.contact-item:nth-child(2) span.contact-item-email').text()
    buyer['email'] = email
    buyer['name'] = html.find('div.main-content-sidebar div.contact-name').text()
    return { 'messages': messages, 'buyer': buyer}

inquiry.get_conversation = types.MethodType(get_conversation, inquiry)

In [15]:
inquiry.load_url()
inquiries = inquiry.get_inquiries()

In [24]:
inquiry.open_inquiry_in_new_tab(inquiries[4])
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
print(conversation)

{'messages': [{'position': 'left', 'user': 'Tiara Green', 'date': '2018-07-01 06:33', 'content': "I'm interested in buying from your store and I would like to request a sample of an eyelash."}, {'position': 'right', 'user': 'Jeremy Wang', 'date': '2018-07-01 10:38', 'content': 'Hi'}], 'buyer': {'email': '隐藏', 'name': 'Tiara Green'}}


In [27]:
def is_new_inquiry(self, inquiry):
    pass